19060374-Kadir Emre Özer-Geri Besleme Algoritması

Algoritmanın ileri besleme kısmı gerçeklenmiştir.Gerçek değer ve hesaplanan nöron değer(3 elemanlı dizi) arasındaki hata hesaplanmış,diğer katmanlara yayılamamıştır.Geri beslemeyi beceremediğim için sadece ilk fotoğraf için ileri besleme yapıp ağın tahminini yazdırdım.

Veri Yollarının belirtilmesi ve yolları ile verilerin bir DataFrame içerisine alınması

In [484]:
import cv2 as cv
import numpy as np
import random as rd
from sklearn import preprocessing #Eğer bir gün "Python iyi bir dil" dersem beni gözünüzü kırpmadan vurunuz.-Nejat İşler 
import pandas as pd 
import os


train_path = './datas/train/' 
test_path = './datas/val/' 


train_df = pd.DataFrame()
test_df = pd.DataFrame()
train_df['images'] = os.listdir(train_path+'/cat') + os.listdir(train_path+'/dog') + os.listdir(train_path+'/wild') 
test_df['images'] = os.listdir(test_path+'/cat') + os.listdir(test_path+'/dog') + os.listdir(test_path+'/wild')

Eğitim verilerinin hazırlanması 

In [485]:
classes = []
paths = []
for image in train_df['images']:
    class_ = image.split('_')[1]
    classes.append(class_)
    paths.append(train_path+'/'+class_+'/'+image)

train_df['classes'] = classes
train_df['path'] = paths
print(train_df)

                        images classes  \
0        flickr_cat_000002.jpg     cat   
1        flickr_cat_000003.jpg     cat   
2        flickr_cat_000004.jpg     cat   
3        flickr_cat_000005.jpg     cat   
4        flickr_cat_000006.jpg     cat   
...                        ...     ...   
14625  pixabay_wild_001249.jpg    wild   
14626  pixabay_wild_001250.jpg    wild   
14627  pixabay_wild_001251.jpg    wild   
14628  pixabay_wild_001252.jpg    wild   
14629  pixabay_wild_001253.jpg    wild   

                                              path  
0         ./datas/train//cat/flickr_cat_000002.jpg  
1         ./datas/train//cat/flickr_cat_000003.jpg  
2         ./datas/train//cat/flickr_cat_000004.jpg  
3         ./datas/train//cat/flickr_cat_000005.jpg  
4         ./datas/train//cat/flickr_cat_000006.jpg  
...                                            ...  
14625  ./datas/train//wild/pixabay_wild_001249.jpg  
14626  ./datas/train//wild/pixabay_wild_001250.jpg  
14627  ./datas/tra

Test verilerinin hazırlanması 

In [486]:
classes = []
paths = []
for image in test_df['images']:
    class_ = image.split('_')[1]
    classes.append(class_)
    paths.append(test_path+'/'+class_+'/'+image)

test_df['classes'] = classes
test_df['path'] = paths
print(test_df)

                       images classes  \
0       flickr_cat_000008.jpg     cat   
1       flickr_cat_000011.jpg     cat   
2       flickr_cat_000016.jpg     cat   
3       flickr_cat_000056.jpg     cat   
4       flickr_cat_000076.jpg     cat   
...                       ...     ...   
1495  pixabay_wild_001165.jpg    wild   
1496  pixabay_wild_001170.jpg    wild   
1497  pixabay_wild_001184.jpg    wild   
1498  pixabay_wild_001192.jpg    wild   
1499  pixabay_wild_001224.jpg    wild   

                                           path  
0        ./datas/val//cat/flickr_cat_000008.jpg  
1        ./datas/val//cat/flickr_cat_000011.jpg  
2        ./datas/val//cat/flickr_cat_000016.jpg  
3        ./datas/val//cat/flickr_cat_000056.jpg  
4        ./datas/val//cat/flickr_cat_000076.jpg  
...                                         ...  
1495  ./datas/val//wild/pixabay_wild_001165.jpg  
1496  ./datas/val//wild/pixabay_wild_001170.jpg  
1497  ./datas/val//wild/pixabay_wild_001184.jpg  
1498  .

In [487]:
images = []
for file in train_df["path"]:
    img = cv.imread(file)
    img = cv.resize(img, (30,30))
    img = img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
    images.append(img)

Katmanların Belirlenmesi ve Katman Nöron Sayıları : giriş 2700 nöron-2 ara katman (270 ve 90 nöron)-çıkış 3 nöron

In [488]:
input_layer = [] # 30*30*3 = 2700
input_layer_size = 30*30*3

hidden_layer_1 = [] # 30*3*3
hidden_layer_1_size = 30*3*3

hidden_layer_2 = [] # 30*3
hidden_layer2_size = 30*3

output_layer = [] # 3
output_layer_size = 3


def load_input_layer(i): #Giriş Katmanını yükle
    global input_layer,images
    input_layer = images[i]/255

Katmanlar Arası Bağların Oluşturulması : Her katman bir diğerine tam bağlıdır(fully-connected).Her katmanın bir diğerine bağı matris şeklinde tanımlanmıştır.Katmanlar arası bağlar ilk başta rastgele atanmıştır.

In [489]:
W_Input_to_Hidden1 = np.random.randn(input_layer_size,hidden_layer_1_size) #input_layer_size x hidden_layer_1_size matrix
W_Hidden1_to_Hidden2 = np.random.randn(hidden_layer_1_size,hidden_layer2_size)
W_Hidden2_to_Output = np.random.randn(hidden_layer2_size,output_layer_size)
#len(W_Hidden2_to_Output) # len(xxx) rows, len(xxx[0]) columns
print(np.shape(W_Input_to_Hidden1))
print(np.shape(W_Hidden1_to_Hidden2))
print(np.shape(W_Hidden2_to_Output))
print(np.shape(input_layer)) # 1x2700

(2700, 270)
(270, 90)
(90, 3)
(0,)


Sigmoid İşlevi

In [490]:
def sigmoid(x:float):
    return (1/(1+np.exp(-x)))

Sigmoid'in Türevi İşlevi

In [491]:
def derivative_sigmoid(x:float):
    return x *(1-x)
    #return ((np.exp(-x)) / ((1 + np.exp(-x))^2))


İleri Besleme İşlevi

In [492]:
def forward():
    global hidden_layer_1,hidden_layer_2,output_layer,input_layer
    hidden_layer_1 = sigmoid(np.dot(input_layer,W_Input_to_Hidden1))
    hidden_layer_2 = sigmoid(np.dot(hidden_layer_1,W_Hidden1_to_Hidden2))
    output_layer = sigmoid(np.dot(hidden_layer_2,W_Hidden2_to_Output))

Yapılamamış Geri Besleme İşlevi

In [493]:
def back_prop(x:int):
    global output_error,hidden2_error,hidden1_error,input_error,output_layer,hidden_layer2,input_layer,W_Hidden2_to_Output,W_Input_to_Hidden1,W_Hidden1_to_Hidden2

    if(train_df.values[x,1] == 'cat'): # Sınıfların Vektörel tanımı
        real = [1,0,0]
    elif(train_df.values[x,1] == 'dog'):
        real = [0,1,0]
    elif(train_df.values[x,1] == 'wild'):
        real = [0,0,1]

    output_error = output_layer - real #ÇIKIŞ KATMANI HATASI 
    output_delta = output_error * derivative_sigmoid(output_layer)
    
    hidden2_error = output_delta.dot(W_Hidden2_to_Output.T) # GİZLİ KATMAN 2 HATASI 
    hidden2_delta = hidden2_error * derivative_sigmoid(hidden_layer_2)

    hidden1_error = hidden2_delta.dot(W_Hidden1_to_Hidden2.T) # GİZLİ KATMAN 1 HATASI 
    hidden1_delta = hidden1_error * (derivative_sigmoid(hidden_layer_1))

    W_Input_to_Hidden1 += input_layer.T.dot(hidden1_delta)
    W_Hidden1_to_Hidden2 += hidden_layer_1.T.dot(hidden2_delta)
    W_Hidden2_to_Output += hidden_layer2.T.dot(output_delta)


Ağın denenmesi(Sadece bir fotoğrafın alınıp ileri besleme yapılması)

In [494]:
output_error,hidden2_error,hidden1_error,input_error = 0,0,0,0
# for i in range (len(train_df)):
#     load_input_layer(i)
#     forward()
#     back_prop(i)
for i in range(1):
    load_input_layer(i)
    forward()
    print(output_layer)


[0.9793512  0.03722832 0.00144655]
